In [1]:
import preprocessing_py as prep
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
from IPython.display import display , Markdown

In [44]:
import os

In [45]:
it_df = pd.read_csv('preprocessing_it.csv')
it_df.shape

(102, 3)

In [48]:
it_df = it_df.iloc[:,1:]
it_df.rename(columns={'0':'raw','1':'tuned'},inplace=True)
it_df['tuned'] = it_df['tuned'].apply(lambda x : x.replace(",",' '))

In [49]:
it_df.tail()

,raw,tuned
97,유영민 과학기술정통부 장관(오른쪽)이 22일 오후 서울 강서구 LG사이언스파크에서 ...,유영 민 과학기술 정통부 장관 오른쪽 오후 서울 강서구 사이언스 파크 중소기업 상생...
98,당당맨 최영수가 시사회 진행해 큰 인기(지디넷코리아=김민선 기자)애니메이션 &apo...,당당 맨 최영 수가 시사회 진행 인기 디 넷 코리아 김 민선 애니메이션 아이스 에이...
99,"전년比 4.9% 증가 '역대 최대'삼성전자, 최다 출원 5761건4차 산업혁명 시대...",전 년 증가 역대 최대 삼성 전자 최 출원 건 차 산업혁명 시대 핵심 자산 지식 재...
100,윤종인 행정안전부 차관(왼쪽 세번째)이 22일 오후 서울 종로구 정부서울청사에서 열...,윤 종인 행정 안전 부 차관 왼쪽 세 오후 서울 종로구 정부 서울 청사 블 록 체인...
101,[전자신문인터넷 박동선기자] 'K팝 명가' SM엔터테인먼트(이하 SM)가 4차 산업...,전자신문 인터넷 박동 선기 팝 명가 엔터테인먼트 이하 차 산업 기술 발전 발 차세대...


In [50]:
# raw documents to tf-idf matrix: 
vectorizer = TfidfVectorizer( use_idf=True, 
                             smooth_idf=True)
# SVD to reduce dimensionality: 
svd_model = TruncatedSVD(n_components=5,
                         algorithm='randomized',
                         n_iter=10)
# pipeline of tf-idf + SVD, fit to and applied to documents:
svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])
# svd_matrix can later be used to compare documents, compare words, or compare queries with documentsa
documents = it_df['tuned'].tolist()
svd_matrix = svd_transformer.fit_transform(documents)

In [51]:
tfidf_encoding_vector = vectorizer.fit_transform(documents)
svd_matrix = svd_model.fit_transform(tfidf_encoding_vector)

In [55]:
print(svd_matrix.shape)
svd_matrix

(102, 5)


array([[ 2.60599236e-01, -3.97212865e-02,  4.20335502e-01,
         1.95781571e-01, -2.52184640e-01],
       [ 1.52032548e-01, -2.55084482e-02,  9.26432889e-03,
        -2.84366300e-02,  5.70541228e-02],
       [ 3.06213303e-01, -1.39739745e-01,  1.24294984e-01,
        -1.09839426e-01, -6.42499768e-02],
       [ 1.75708171e-01, -1.69289078e-02,  1.55163883e-01,
        -2.03451478e-01,  6.06721707e-02],
       [ 6.55834853e-02, -2.15937286e-02,  5.01504418e-02,
        -6.84057794e-02,  1.97128119e-01],
       [ 1.39988352e-01,  1.32835074e-02,  8.65993354e-02,
        -1.49371478e-01,  4.32390491e-02],
       [ 6.80485155e-02, -1.60572200e-02, -3.16402820e-03,
        -1.10582029e-02,  6.18490235e-01],
       [ 2.22799750e-01, -4.30826378e-02,  1.60023829e-01,
        -1.87357842e-01,  3.15073843e-01],
       [ 9.37675024e-02, -2.89161339e-02,  5.00652161e-02,
        -1.07601510e-01,  8.38490596e-02],
       [ 4.61334048e-01,  7.24005787e-01, -1.19768114e-01,
         3.91726459e-02

In [56]:
cluster_ls = []
for idx in range(len(svd_matrix)) :
    cluster_ls.append((it_df['raw'][idx],np.argmax(svd_matrix[idx]),np.max(svd_matrix[idx])))

In [60]:
print(cluster_ls[0][0])

배정회 신임 과학기술일자리진흥원장[아시아경제 이민우 기자] 배정회 전 국립중앙과학관전시연구단장(56)이 과학기술일자리진흥원 제3대 원장으로 취임했다.과학기술정보통신부는 22일 이 같은 인사를 발표했다. 임기는 이날부터 3년 동안이다.배 신임 원장은 행정고등고시 40회로 공직에 입문했다. 교육과학기술부 융합기술과장, 중앙대 경영경제대 초빙교수, 과기정통부연구성과활용정책과장 및 연구성과혁신기획과장 등을 거쳤다. 과학기술 정책 수립부터 정부 연구개발(R&D;), 연구성과의 기술 이전 및 사업화까지 두루 경험을 보유한 것으로 평가받는다.배 신임 원장은 이날 환담회에서 "새로운 임무에 맞게 조직을 신속히 재구성하고 직원 역량을 강화하겠다'며 "실험실 유망기술을 R&D; 전문기업과 연결하며 기술이전을 효과적으로 지원하고 신기술창업을 통한 미래형 일자리를 많이 창출시키겠다"고 밝혔다.이민우 기자 letzwin@asiae.co.kr▶ 네이버 홈에서 '아시아경제' 뉴스 확인하기▶ 재미와 신기 '과학을읽다' ▶ 꿀잼 '인기만화'<ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제 무단전재 배포금지>


In [62]:
print(cluster_ls[0][1])

2


In [54]:
cluster_ls = []
for idx in range(len(svd_matrix)) :
    cluster_ls.append((it_df['raw'][idx],np.argmax(svd_matrix[idx]),np.max(svd_matrix[idx])))

new_ls = [i[0] for i in cluster_ls]
cat_ls = [i[1] for i in cluster_ls]
score_ls = [i[2] for i in cluster_ls]

lsa_df = pd.DataFrame()
lsa_df['news'] = new_ls
lsa_df['cat'] = cat_ls
lsa_df['score'] = score_ls

for i in range(5) :
    if \
    len(lsa_df[lsa_df['cat'] == i].sort_values('score',ascending=False).head(5)) == 0 :
        display(Markdown('### There is no data which belong to {}-th cat!'.format(i+1)))

    else :display(lsa_df[lsa_df['cat'] == i].sort_values('score',ascending=False).head(5))

,news,cat,score
74,"LG 사이언스파크에서 5G 준비현황 점검 후 업계 의견 청취유영민 장관 ""5G시대 ...",0,0.655825
61,"[머니투데이 김주현 기자] [3개부처 장관, LG 5G 현장방문…유영민 장관 ""통신...",0,0.626954
64,LG유플러스가 준비 중인 5G 서비스에 대한 전시와 시연에서 성윤모 산업통상자원부장...,0,0.594890
45,"유영민 과학기술정보통신부장관(가운데)과 성윤모 산업통상자원부장관(왼쪽), 홍종학 중...",0,0.565521
65,"과기·산업·중기장관 등 마곡 방문유영민(왼쪽 세번째) 과학기술정보통신부 장관, 성윤...",0,0.550061


,news,cat,score
9,"KT스카이라이프 ""세부 내용은 확인 필요""국회 과방위, 합산규제 문제 2월 재논의【...",1,0.724006
22,과기정통부.KT 공공성 회복 방안 검토 후 다시 논의【서울=뉴시스】이국현 기자 = ...,1,0.723300
86,"김성태 의원 ""KT스카이라이프 공공성 확보 방안, 과기정통부에 요청…합산규제 재도입...",1,0.703520
14,"과방위, 규제 유지 쪽으로 기울어향후 M&A; 등 사업확장 걸림돌22일 국회 과학기...",1,0.699562
62,[아시아경제 명진규 기자] 국회 과학기술정보방송통신위원회(이하 과방위)는 22일 법...,1,0.695846


,news,cat,score
50,[이데일리 이연호 기자] 과학기술정보통신부는 배정회(57·사진) 전 국립중앙과학관 ...,2,0.473784
27,배정회 과기일자리진흥원장배정회 과학기술일자리진흥원 원장(앞줄 왼쪽 네번째)이 22일...,2,0.437691
0,배정회 신임 과학기술일자리진흥원장[아시아경제 이민우 기자] 배정회 전 국립중앙과학관...,2,0.420336
52,[이데일리 이연호 기자] △정책부장 김강회 △기획부장 송장헌 △연구데이터공유센터장 ...,2,0.350561
33,"석제범 IITP 원장 간담회석제범 정보통신기획평가원장""이노베이션 아카데미, AI대학...",2,0.308470


,news,cat,score
49,[이데일리 이연호 기자]◇직책승진(8명)△감사부 부장 변재규△창의융합교육단 단장 허...,3,0.684312
63,<직책 승진>Δ변재규 감사부 부장 Δ허경호 창의융합교육단 단장Δ 김명수 과학문화산업...,3,0.673451
15,◇한국과학창의재단 ◇승진 △감사부장 변재규 △창의융합교육단장 허경호(종합교육연수원장...,3,0.656683


,news,cat,score
6,동영상 뉴스,4,0.618490
38,동영상 뉴스,4,0.618490
36,(서울=연합뉴스) 성윤모 산업통상자원부 장관이 22일 오후 서울 강서구 마곡동 LG...,4,0.406159
37,(서울=연합뉴스) 성윤모 산업통상자원부 장관이 22일 오후 서울 강서구 마곡동 LG...,4,0.406159
7,"[앵커] 검색 기록과 내비게이션 이동경로까지, 일상 속 수많은 정보가 쌓여 '빅 ...",4,0.315074
